### Annual Rankings of Streamflow, SWE, ET, Soil Moisture, Precip (with anomalies)

This notebook will first build a linear regression between streamflow and SWE anomalies and compute the residual for these years and rank them. Then I will do the same for ranking years with annual ET signals, seasonal ET signals, winter, spring, and summer precip signals and fall soil moisture signals. Then I will look at how these years align with each other. 

This will cover the Water Years 1987 to 2020

In [71]:
import pandas as pd
import numpy as np
import xarray as xr

import datetime as dt

import rioxarray as rioxr
from rasterio.enums import Resampling
import geopandas as gpd 
import fiona
fiona.drvsupport.supported_drivers['KML'] = 'rw'

import seaborn as sns
import matplotlib.pyplot as plt
import xoak
import nctoolkit as nc
from dataretrieval import nwis
from scipy import stats


In [ ]:
# Polygon for Upper East River
upper_east_river_poly = gpd.read_file('./multisite/polygons/east_polygon.json')
upper_east_river_area = upper_east_river_poly.area # m^2

### Pull in Snotel data and streamflow data

In [2]:
# Climatology including precipitation, snowfall and temperature for each month
bb_climatology = pd.read_csv('../data/billy_barr_monthly_avg.csv',sep='\t')
bb_climatology[bb_climatology==" "] = np.nan
bb_climatology['WY'] = [int(str(year)[0:2]+str(year)[4:]) for year in bb_climatology['Year']]
bb_climatology['water_cm'] = bb_climatology['water_cm'].astype(float)
dates = pd.to_datetime([f"{bb_climatology.loc[i,'WY']}-{bb_climatology.loc[i,'Month']}-01" for i in bb_climatology.index], format='%Y-%m-%d')
date_list=[]

for i,date in enumerate(dates):
    if date.year == 1900:
        date = dt.date(2000,date.month,date.day)
    if date.month in [9,10,11,12]:
        date = dt.date(date.year-1,date.month,date.day)
    else:
        date = dt.date(date.year,date.month,date.day)
    date_list.append(date)
bb_climatology.index=date_list
bb_climatology = bb_climatology.drop(['Month', 'Year'], axis=1)
for col in bb_climatology:
    bb_climatology[col] = bb_climatology[col].astype(float)

In [3]:
# Import snotel data
sntl_ds = xr.open_dataset('/storage/dlhogan/sos/data/east_river_sntl_20220930.nc')

In [108]:
er_usgs_almont = '09112500'

# get streamflow data from east river at almont
east_river_discharge_almont_dv = nwis.get_record(er_usgs_almont, start='1986-10-1', service='dv')['00060_Mean']

# Filter out until the end of our dataset
east_river_discharge_almont_dv = east_river_discharge_almont_dv.loc[:'2020-09-30']
# Filter out -9999 values
east_river_discharge_almont_dv = east_river_discharge_almont_dv[east_river_discharge_almont_dv>=0]

# Average to monthly values
east_river_discharge_almont_monthly = east_river_discharge_almont_dv.groupby(pd.Grouper(freq='M')).mean()
east_river_discharge_almont_monthly.index = pd.to_datetime(east_river_discharge_almont_monthly.index.date)

# Set index as datetime
east_river_discharge_almont_dv.index = pd.to_datetime(east_river_discharge_almont_dv.index).date

# Rename the series
east_river_discharge_almont_monthly.name = 'mean_discharge'
# Convert to dataframe and add in water year column
east_river_discharge_almont_monthly = east_river_discharge_almont_monthly.to_frame()
east_river_discharge_almont_monthly['WY'] = east_river_discharge_almont_monthly.index.year.where(east_river_discharge_almont_monthly.index.month < 10, east_river_discharge_almont_monthly.index.year + 1)

# create annual product
east_river_discharge_almont_annual = east_river_discharge_almont_monthly.groupby(east_river_discharge_almont_monthly['WY']).mean()

# create seasonal product 

# Wrap it into a simple function
def season_mean(ds, calendar="standard"):
    # Make a DataArray with the number of days in each month, size = len(XTIME)
    month_length = ds.index.dt.days_in_month

    # Calculate the weighted average
    return (ds * month_length).resample(index='QS-DEC').sum() / month_length.resample(index='QS-DEC').sum()

east_river_discharge_almont_monthly_ds = east_river_discharge_almont_monthly.to_xarray()
east_river_discharge_almont_seasonal_ds = season_mean(east_river_discharge_almont_monthly_ds)
east_river_discharge_almont_seasonal_ds['WY'] = np.ceil(east_river_discharge_almont_seasonal_ds['WY'])

### Linear relationship between Streamflow and SWE

In [109]:
# normalized streamflow
normalized_er_annual_q = (east_river_discharge_almont_annual - east_river_discharge_almont_annual.mean())/ east_river_discharge_almont_annual.std()

# normalized peak SWE
# create mean dataset for schofield and butte
er_sntl_data = xr.open_dataset('../../../../../storage/dlhogan/sos/data/east_river_sntl_20220930.nc')
butte_schofield_mean_ds = er_sntl_data.sel(Location=['Butte_380:CO:SNTL','SchofieldPass_737:CO:SNTL']).mean(dim='Location')
butte_schofield_mean_ds_filtered = butte_schofield_mean_ds.sel(Date=slice('1986-10-01', '2020-09-30'))
butte_schofield_mean_peak_swe = butte_schofield_mean_ds_filtered.WTEQ.groupby(butte_schofield_mean_ds_filtered.WY).max()
normalized_sntl_peak_swe = (butte_schofield_mean_peak_swe - butte_schofield_mean_peak_swe.mean())/butte_schofield_mean_peak_swe.std()

In [120]:
slope, intercept, rvalue, pvalue, stderr = stats.linregress()

In [123]:
slope, intercept, rvalue, pvalue, stderr = stats.linregress(normalized_sntl_peak_swe.values, normalized_er_annual_q.values.T)

alpha = 0.05
c = (1 - alpha)

# Create regression line and residuals
X = normalized_sntl_peak_swe
y_predicted = intercept + slope*X
residual = (normalized_er_annual_q.values-y_predicted)

# Setup 
n = X.size                                               
dof = n - 2
t = stats.t.ppf(c, dof) 

# sum of squared errors
sse = np.sum(residual**2)

# total sum of squares (y)
sst = np.sum( (normalized_er_annual_q - np.mean(normalized_er_annual_q))**2 )

# total sum of squares (x)
sst_x = np.sum( (X - np.mean(X))**2 )

# correlation coefficient
r_squared = 1 - sse/sst

# standard error of regression
s = np.sqrt(sse/(n-2))

# an array of x values
p_x = np.linspace(X.min(),X.max(),100)

# using our model parameters to predict y values
p_y = intercept + slope*p_x

# compute error of prediction for each p_x
sigma_ep = np.sqrt( s**2 * (1+ 1/n + ( ( n*(p_x-X.mean())**2 ) / ( n*np.sum(X**2) - np.sum(X)**2 ) ) ) )
# set our confidence interval

p_y_lower = p_y - t * sigma_ep
p_y_upper = p_y + t * sigma_ep


ValueError: applied function returned data with unexpected number of dimensions. Received 2 dimension(s) but expected 1 dimensions with names: ('WY',)

In [124]:
y_predicted

<xarray.DataArray 'WTEQ' (WY: 34)>
array([ 2.55997837e-01, -6.82035293e-01,  7.50726795e-04, -8.35183560e-01,
       -1.58778717e-01, -9.94713004e-01,  1.47480279e+00, -3.24689339e-01,
        2.03634643e+00,  9.26021502e-01,  1.06640741e+00, -2.54496384e-01,
       -1.14110473e-01, -6.24604694e-01, -4.84218783e-01, -9.50044759e-01,
       -5.35268205e-01, -8.03277671e-01,  6.19724969e-01,  2.24091948e-01,
       -8.35183560e-01,  1.72366872e+00,  8.04779124e-01, -4.39175175e-02,
        1.25784275e+00, -1.35844014e+00, -6.82035293e-01,  3.58096681e-01,
       -1.16700480e+00, -8.28802382e-01,  1.17488743e+00, -9.05376515e-01,
        1.10469448e+00, -4.45931716e-01])
Coordinates:
  * WY       (WY) int64 1987 1988 1989 1990 1991 ... 2016 2017 2018 2019 2020

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(20,10))
colors = ['#2c7fb8','#7fcdbb']
residual=residual.dropna()
idx_25th_percentile = (residual[residual < np.percentile(residual,10)].index)
idx_95th_percentile = residual[residual > np.percentile(residual,90)].index


axs[0].scatter(normalized_sntl_peak_swe, normalized_er_annual_q, color=colors[0], edgecolor='k', s=60, lw=2)
axs[0].axhline(0, color='k', lw=2)
axs[0].axvline(0, color='k', lw=2)
axs[0].plot([X.min(),X.max()], intercept+slope*np.array([X.min(),X.max()]), color=colors[0], label='Best-Fit Line')
axs[0].plot([-1.5,2.5],[-1.5,2.5], label='1-to-1 line', color='grey', ls='--', alpha=0.6)
# axs[0].plot(p_x, p_y_lower, color=colors[1], ls='--', label='Upper/Lower 90% Confidence Bound')
# axs[0].plot(p_x, p_y_upper, color=colors[1], ls='--', )
# axs[0].fill_between(p_x, p_y_lower,p_y_upper, color=colors[1], alpha=0.3)

# axs[1].scatter(normalized_sntl_peak_swe,residual.loc[1992:],color=colors[0],edgecolor='k', s=60, lw=2);
# axs[1].axhline(0, color='k', lw=2)
# axs[1].axvline(0, color='k', lw=2)
# axs[1].set_xlabel('APR + MAY Precip Normalized Anomaly')
# axs[1].set_ylabel('SWE-Discharge Residual',)
# axs[1].set_title('(b) SWE-Discharge Residual and Apr+May Precip Anomaly')


for i,idx in enumerate(idx_25th_percentile):
    if idx in [2012,2017,2018]:
        axs[0].annotate(str(idx), 
                    (east_annual_post1992_norm.loc[idx], east_annual_post1992_norm.loc[idx]),
                    (east_annual_post1992_norm.loc[idx]+0.2, east_annual_post1992_norm.loc[idx]-0.4),
                    arrowprops=dict(facecolor='blue', shrink=0.1),
                    horizontalalignment='center', verticalalignment='top',)
    elif idx in [2021]:
        axs[0].annotate(str(idx), 
                        (butte_sntl_peaks_post1992_norm.loc[idx], east_annual_post1992_norm.loc[idx]),
                        (butte_sntl_peaks_post1992_norm.loc[idx]+0.4, east_annual_post1992_norm.loc[idx]-0.2),
                        arrowprops=dict(facecolor='blue', shrink=0.1),
                        horizontalalignment='center', verticalalignment='top',)
        axs[1].annotate(str(idx), 
                (spring_precip_norm_anomaly.loc[idx], residual.loc[idx]),
                (spring_precip_norm_anomaly.loc[idx]-0.2, residual.loc[idx]-0.1),
                arrowprops=dict(facecolor='blue', shrink=0.1),
                horizontalalignment='center', verticalalignment='top',)
                
    else:
        axs[0].annotate(str(idx), 
                        (butte_sntl_peaks_post1992_norm.loc[idx], east_annual_post1992_norm.loc[idx]),
                        (butte_sntl_peaks_post1992_norm.loc[idx]+0.4, east_annual_post1992_norm.loc[idx]-0.2),
                        arrowprops=dict(facecolor='blue', shrink=0.1),
                        horizontalalignment='center', verticalalignment='top',)
        axs[1].annotate(str(idx), 
                (spring_precip_norm_anomaly.loc[idx], residual.loc[idx]),
                (spring_precip_norm_anomaly.loc[idx]+0.5, residual.loc[idx]),
                arrowprops=dict(facecolor='blue', shrink=0.1),
                horizontalalignment='center', verticalalignment='top',)

for i,idx in enumerate(idx_95th_percentile):
    if idx == 2001:
        axs[0].annotate(str(idx), 
                (butte_sntl_peaks_post1992_norm.loc[idx], east_annual_post1992_norm.loc[idx]),
                (butte_sntl_peaks_post1992_norm.loc[idx], east_annual_post1992_norm.loc[idx]-0.4),
                arrowprops=dict(facecolor='red', shrink=0.1),
                horizontalalignment='center', verticalalignment='top',)
        
    else:
        axs[0].annotate(str(idx), 
                (butte_sntl_peaks_post1992_norm.loc[idx], east_annual_post1992_norm.loc[idx]),
                (butte_sntl_peaks_post1992_norm.loc[idx]-0.4, east_annual_post1992_norm.loc[idx]),
                arrowprops=dict(facecolor='red', shrink=0.1),
                horizontalalignment='center', verticalalignment='top',)
        axs[1].annotate(str(idx), 
                (spring_precip_norm_anomaly.loc[idx], residual.loc[idx]),
                (spring_precip_norm_anomaly.loc[idx], residual.loc[idx]-0.2),
                arrowprops=dict(facecolor='red', shrink=0.1),
                horizontalalignment='center', verticalalignment='top',)

axs[0].arrow([],[],[],[], color='red', label='Years above 85th perctile',width=0.1)
axs[0].arrow([],[],[],[], color='blue', label='Years below 15th percentile',width=0.1)
axs[0].set_xlabel('Normalized Peak SWE')
axs[0].set_ylabel('Normalized Annual Mean Discharge')
axs[0].set_title('(a) Butte Peak SWE against East River Mean Flow')
axs[0].legend(edgecolor='k', facecolor='white')


